In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("./..")
from PARC.data import data_navier_stokes as data
from PARC.model import model_ns as model
from PARC import misc


2024-01-18 11:09:56.333802: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data pipeline

In [2]:
Re_list = [15,20,30,40,60,80,100,120,140,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
train_list = [30,40,80,100,150,200,250,300,400,450,500,600,650,700,800,850,900,950]
test_list = [20,60,140,350,550,750,1000]

data_ns = data.DataNS(Re_list)
seq_input = data_ns.import_data(folder_path='./data/ns/')
seq_norm = data_ns.data_normalization(input_data=seq_input, no_of_channel=3)

train_seq, test_seq = data_ns.train_test_split(Re_list=Re_list, Re_test=test_list, Re_train=train_list,input_data=seq_norm[0])
train_data = data_ns.clip_data(input_seq=train_seq,no_of_fields=3,sequence_length=13)
# def clip_raw_data():
#     data_whole = []
#     # r_whole = []
#     for Re in Re_list:
#         data_file_name = 'Re_' + str(int(Re)) + '.npy'
#         file_path = './data/ns/' + data_file_name                
#         if os.path.exists(file_path):
#             raw_data = np.float32(np.load(file_path))
#             raw_data = np.expand_dims(raw_data, axis = 0)
#             data_whole.extend(raw_data)
#     data_whole = np.concatenate([data_whole], axis=0)
#     return data_whole
# seq_clipped = clip_raw_data()

In [3]:
# # Normalization
# def data_normalization(input_data,no_of_channel):
#     norm_data = np.zeros(input_data.shape)
#     min_val = []
#     max_val = []
#     for i in range(no_of_channel):
#         iter_max_val = np.amax(input_data[:,:,:,i::3])
#         iter_min_val = np.amin(input_data[:,:,:,i::3])
#         norm_data[:,:,:,(i)::no_of_channel] = ((input_data[:,:,:,(i)::no_of_channel] - iter_min_val)) / (iter_max_val - iter_min_val + 1E-9)
#         min_val.append(iter_min_val)
#         max_val.append(iter_max_val)
#     return norm_data, min_val, max_val
# seq_norm = data_normalization(seq_clipped, 3)

In [6]:
# Re_list = [15,20,30,40,60,80,100,120,140,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]
# train_list = [30,40,80,100,150,200,250,300,400,450,500,600,650,700,800,850,900,950]
# test_list = [20,60,140,350,550,750,1000]
# idx = 0
# train_idx =[]
# test_idx =[]
# for Re in Re_list:
#     if Re in train_list:
#         train_idx.append(idx)
#     elif Re in test_list:
#         test_idx.append(idx)
#     idx += 1

In [7]:
# train_seq = [seq_norm[0][idx:idx+1,:,:,:] for idx in train_idx]
# test_seq = [seq_norm[0][idx:idx+1,:,:,:] for idx in test_idx]
# train_seq = np.concatenate(train_seq, axis = 0)
# test_seq = np.concatenate(test_seq, axis = 0)
# print(test_seq.shape)

In [12]:
# def create_train_data(seq, no_of_fields, sequence_length = 2):
#     shape = seq.shape
#     num_time_steps = np.int32((shape[-1]-1)/3)
#     vel_seq_whole = []
#     for i in range(shape[0]):
#         for j in range(num_time_steps-sequence_length+1):
#             vel_seq_case = np.expand_dims(seq[i, :, :, (j*no_of_fields):(j*no_of_fields+sequence_length*no_of_fields)],axis = 0)
#             vel_seq_whole.extend(vel_seq_case)
#     vel_seq_whole = np.concatenate([vel_seq_whole], axis=0)
#     return vel_seq_whole
# train_data = create_train_data(train_seq, no_of_fields = 3, sequence_length = 13)

# Training


### Stage 1: Differentiator training

In [13]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(train_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 798) 
dataset = dataset.batch(4)

2024-01-06 14:52:11.572250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78791 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "differentiator_training")
# parc.differentiator.load_weights('parc2_diff_ns_heun_10_2.h5')
# parc.poisson.load_weights('parc2_poisson_ns_heun_10_2.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 500, shuffle = True)

parc.differentiator.save_weights('PARC2_NS_diff_heunn12.h5')
parc.poisson.save_weights('PARC2_NS_diff_poisson_heunn12.h5')

### Stage 2: Integrator training (Can skip)

In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 12, step_size= 1/38, solver = "heun", mode = "integrator_training")
parc.differentiator.load_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_12.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001, beta_1 = 0.5, beta_2 = 0.999))
parc.fit(dataset, epochs = 300, shuffle = True)

parc.integrator.save_weights('PARC2_NS_int_heunn12.h5')

# Validation

In [16]:
tf.keras.backend.clear_session()
parc = model.PARCv2_ns(n_time_step = 37, step_size= 1/38, solver = "heun", use_data_driven_int = False, mode = "differentiator_training")

misc.geturl(url="https://virginia.box.com/shared/static/u6msftownrqjzlwhysmm24y4bxc11pdr", title="PARC2_NS_int_heunn12.h5")
misc.geturl(url="https://virginia.box.com/shared/static/edusf1diccn8yle0n9k90w5he4zdma54", title="PARC2_NS_diff_heunn12.h5")
misc.geturl(url="https://virginia.box.com/shared/static/cyhbtqanwwz0msdod2robstatlvk9faa", title="PARC2_NS_diff_poisson_heunn12.h5")

parc.differentiator.load_weights('parc2_diff_ns_heun_12.h5')
parc.poisson.load_weights('parc2_poisson_ns_heun_12.h5')
parc.integrator.load_weights('parc2_int_ns_heun_10.h5')
parc.compile()

In [10]:
pred_whole =[]
for idx in range(7):
    input_seq_current = tf.cast(test_seq[idx:idx+1,:,:,:3], dtype = tf.float32)
    output = parc.predict(input_seq_current)
    pred_whole.append(output)
pred = np.concatenate(pred_whole,axis = 0)

2024-01-07 10:45:56.534022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-01-07 10:45:57.343827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 248ms/step


In [ ]:
y_pred_denorm = data_ns.data_denormalization(pred,seq_norm[1], seq_norm[2], no_of_channel = 3)

In [74]:
np.save('./plotting/ns/parcv2_ns.npy',y_pred_denorm)

## Visualizing result

In [ ]:
# Load result data
gt = np.load('../result/ns/gt.npy')
# NeuralODE
parc = np.load('../result/ns/parc_ns.npy')
# NeuralODE
parc_data = np.load('../result/ns/parc_ns_data_int.npy')
# FNO
fno = np.load('../result/ns/fno_ns.npy')
# PhyCRNet
phycrnet = np.load('../result/ns/phycrnet_ns.npy')
# PI-FNO
pifno = np.load('../result/ns/pifno_ns.npy')
# PARCv2
parcv2 = np.load('../result/ns/parcv2_ns_new.npy')



In [ ]:
# Test Re list
Re_list = [20, 60, 140, 350, 550, 750, 1000]

def turn_off_axis(ax):
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_yticklabels([])
    ax.set_xticklabels([])

In [ ]:
# Normalize the pressure field
for Re in Re_list:
    gt[:,:,:,2::3] /= Re*4
    parc[:,:,:,2::3] /= Re*4
    fno[:,:,:,2::3] /= Re*4
    phycrnet[:,:,:,2::3] /= Re*4
    pifno[:,:,:,2::3] /= Re*4
    parc_data[:,:,:,2::3] /= Re*4

In [ ]:
y_pred = [parcv2, parc, parc_data, fno, phycrnet, pifno]

In [ ]:
# Masking the cylinder area
rr = 0.125 + 1.0/128
#rr = 0.0
x, y = np.linspace(0.0, 1.0, 128), np.linspace(0.0, 2.0, 256)
xx, yy = np.meshgrid(x, y)
mask = (((xx-0.5)**2+(yy-0.5)**2)<=rr*rr)

expanded_mask = np.expand_dims(np.expand_dims(mask.transpose(), axis=0), axis=-1)

# Set corresponding elements in train_data to 0
for i in range(6):
    y_pred[i] = np.where(expanded_mask, np.zeros_like(y_pred[i]), y_pred[i])
gt = np.where(expanded_mask, np.zeros_like(gt), gt)

In [ ]:
# Plot field evolution
from PARC.visualization import visualization_navier_stokes as viz
viz.plot_field_evolution(y_pred, gt, case_id = 4) 

In [ ]:
# Plot MSE, PDE residual graph
dt = 2.0/38            # Temporal resolution
dx = 1.0/128           # Spatial resolution
from PARC import metrics
loss_func = metrics.NSPdeLoss(dt, dx)

def rmse(pred,gt):
    return np.sqrt(np.mean((pred-gt)**2))

def get_loss_pinn(gt,pred):
    rr = 0.125 + 1.0/128
    #rr = 0.0
    x, y = np.linspace(0.0, 1.0, 128), np.linspace(0.0, 2.0, 256)
    xx, yy = np.meshgrid(x, y)
    mask = (((xx-0.5)**2+(yy-0.5)**2)<=rr*rr)
    # print(mask)

    dt = 2.0/38            # Temporal resolution
    dx = 1.0/128           # Spatial resolution
    # loss_func = loss_generator(dt, dx)
    mse_pinn = []
    pde_pinn = []
    div_pinn = []
    # print(pinn.shape)
    for i, Re in enumerate(Re_list):
        # Ground truth
        i_gt = gt[i,:,:,:].reshape(128, 256, 39, 3).transpose(2, 0, 1, 3)[:38, :, :, :]
        u_gt = np.sqrt(i_gt[:,:,:,0:1]**2+i_gt[:,:,:,1:2]**2)

        # # PhyCRNet
        i_pinn = pred[i,:,:,:].reshape(128, 256, 38, 3).transpose(2, 0, 1, 3)[:38, :, :, :]
        # i_pinn[:,:,:,2:] /= (4.0*Re)
        u_pred = np.sqrt(i_pinn[:,:,:,0:1]**2+i_pinn[:,:,:,1:2]**2)
        # np.load("data/phycrnet/Re_%i.npy" % (Re))

        # MSE loss
        mse_pinn_Re = rmse(u_pred,u_gt)
        mse_pinn.append(mse_pinn_Re)
        # PDE loss
        i_pinn = torch.FloatTensor(i_pinn.transpose(0, 3, 2, 1))
        loss_func.set_data(i_pinn)
        fu_pinn, fv_pinn, fp_pinn = loss_func.ComputePdeLoss(4.0 * Re, 1.0/(4.0 * Re), mask)
        fu_pinn = fu_pinn.numpy()
        fv_pinn = fv_pinn.numpy()
        fp_pinn = fp_pinn.numpy()
        pde_pinn.append(np.mean(np.sqrt(fu_pinn * fu_pinn + fv_pinn * fv_pinn)))
        div_pinn.append(np.mean(fp_pinn))

    mse_pinn = np.array(mse_pinn)
    pde_pinn = np.array(pde_pinn)
    div_pinn = np.array(div_pinn)
    return mse_pinn, pde_pinn, div_pinn

mse_whole = []
pde_whole = []
div_whole = []
for i in range(6):
    i_mse_pinn, i_pde_pinn, i_div_pinn = get_loss_pinn(gt, y_pred[i])
    mse_whole.append(i_mse_pinn)
    pde_whole.append(i_pde_pinn)
    div_whole.append(i_div_pinn)
    print(i)

i_mse_pinn, i_pde_pinn, i_div_pinn = get_loss_pinn(gt, gt)

In [ ]:
viz.pde_mse_div_Re(mse_whole,pde_whole,div_whole,Re_list)